In [1]:
import torch

import datasets
import numpy as np
import pandas as pd
import tqdm.notebook as tq



In [2]:
# create data file from retrieal_dataset.ipynb


train_df = pd.read_csv("proto_train_comb_df.csv")
# train_df = train_df.iloc[:100]
train_df.head(), len(train_df)

(   Unnamed: 0 기업 이름                                               news  \
 0           0    남성  중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...   
 1           1    남성  중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...   
 2           2    남성  중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...   
 3           3    남성  중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...   
 4           4    남성  중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...   
 
                                                  psg  
 0  지배기업  남성은 1965년 6월 10일 설립되어 디지털 전자제품 및 자동차 관련 ...  
 1  부동산 임대 사업이 된 사업이나 새로운 사업을 추진하고자 검토중에 있습니다  가  ...  
 2   나타내었습니다  컴퓨터 및 변기기 부문도 컴퓨터의 수출 증가로 전년 동기대비16 ...  
 3  동지역은 8  감소하는 실적을 나타내었습니다  수입은 아시아지역이 21  증가하였으...  
 4  입니다  당사의 수출품목인 모바일미디어 부문도 오디오 비디오 기능의 복합화와 함께 ...  ,
 3219)

In [3]:
valid_df = pd.read_csv("proto_valid_comb_df.csv")
valid_df.head()

,Unnamed: 0,기업 이름,news,psg
0,0,KMH,[ 임근호 기자 ] @CORP@ 와 미국 알파벳 구글 지회사 의 간결한 지배구조를 ...,가 업계현황 및 전망 (1) 방송산업 전반에 관한 사항 (가) 방송산업의 개념 및...
1,1,KMH,[ 임근호 기자 ] @CORP@ 와 미국 알파벳 구글 지회사 의 간결한 지배구조를 ...,사업 종합유선방송사업 위성방송사업 방송채널사용사업 등 4종류로 분류하고 있으며 (법...
2,2,KMH,[ 임근호 기자 ] @CORP@ 와 미국 알파벳 구글 지회사 의 간결한 지배구조를 ...,편되어 IPTV와 UHD방송과 같은 새로운 디지털방송서비스가 등장하면서 빠르게 성장...
3,3,KMH,[ 임근호 기자 ] @CORP@ 와 미국 알파벳 구글 지회사 의 간결한 지배구조를 ...,한 기술개발 노하우와 경쟁력을 가지기 위해서는 최소 3 5년 간의 지속적인 투자를 ...
4,4,KMH,[ 임근호 기자 ] @CORP@ 와 미국 알파벳 구글 지회사 의 간결한 지배구조를 ...,송출서비스시장은 위성방송 외 IPTV 및 기존 케이블TV 등 신규 유료방송 플랫폼의...


In [23]:
# 기업 이름을 label으로 만든다. 나중에 table 넘겨서 pooling에 사용할 것
dic = {}
idx = 0
for c in train_df['기업 이름']:
    if c not in dic:
        dic[c] = idx
        idx += 1

val_dic = {}
idx = 0
for c in valid_df['기업 이름']:
    if c not in val_dic:
        val_dic[c] = idx
        idx += 1

valid_df['corp_code'] = valid_df['기업 이름'].map(lambda x:val_dic[x])
train_df['corp_code'] = train_df['기업 이름'].map(lambda x:dic[x])
train_df.head()

,Unnamed: 0,기업 이름,news,psg,corp_code
0,0,남성,중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...,지배기업 남성은 1965년 6월 10일 설립되어 디지털 전자제품 및 자동차 관련 ...,0
1,1,남성,중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...,부동산 임대 사업이 된 사업이나 새로운 사업을 추진하고자 검토중에 있습니다 가 ...,0
2,2,남성,중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...,나타내었습니다 컴퓨터 및 변기기 부문도 컴퓨터의 수출 증가로 전년 동기대비16 ...,0
3,3,남성,중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...,동지역은 8 감소하는 실적을 나타내었습니다 수입은 아시아지역이 21 증가하였으...,0
4,4,남성,중국 투자의향기업 직접 방문 투자협약 체결 협의 이성훈 여수광양항만공사 사장 방희석...,입니다 당사의 수출품목인 모바일미디어 부문도 오디오 비디오 기능의 복합화와 함께 ...,0


In [5]:
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel
)
import transformers

In [6]:
import transformers
print(transformers.__version__)

4.13.0


In [7]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [24]:
dart_tok = tokenizer( list(train_df['psg']) ,padding = "max_length", return_tensors = "pt", truncation = True) 
news_tok = tokenizer( list(train_df['news']) ,padding = "max_length", return_tensors = "pt", truncation = True) 

dart_valid_tok = tokenizer( list(valid_df['psg']) ,padding = "max_length", return_tensors = "pt", truncation = True) 
news_valid_tok = tokenizer( list(valid_df['news']) ,padding = "max_length", return_tensors = "pt", truncation = True) 


In [25]:
corp_code = torch.tensor(train_df['corp_code'])
valid_corp_code = torch.tensor(valid_df['corp_code'])

In [10]:
dart_tok['input_ids'].shape, news_tok['input_ids'].shape, corp_code.shape

(torch.Size([3219, 512]), torch.Size([3219, 512]), torch.Size([3219]))

In [11]:
dart_tok.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [12]:
from torch.utils.data import DataLoader, TensorDataset

In [27]:
train_set = TensorDataset(
    news_tok['input_ids'], news_tok['token_type_ids'], news_tok['attention_mask'], \
    dart_tok['input_ids'], dart_tok['token_type_ids'], dart_tok['attention_mask'], \
    corp_code)

valid_set = TensorDataset(
    news_valid_tok['input_ids'], news_valid_tok['token_type_ids'], news_valid_tok['attention_mask'], \
    dart_valid_tok['input_ids'], dart_valid_tok['token_type_ids'], dart_valid_tok['attention_mask'], \
    valid_corp_code)

In [28]:
BATCH_SIZE = 16
dataloader = DataLoader(train_set, batch_size=BATCH_SIZE)

val_dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE)

dataloader

# 모델

In [15]:
class BertEncoder(BertPreTrainedModel):

    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
      
    def forward(self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [17]:
news_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)
dart_encoder = BertEncoder.from_pretrained("klue/bert-base").to(device)




Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.tr

# 학습

In [18]:
from torch import optim
news_optimizer = optim.SGD(news_encoder.parameters(), lr=0.01, momentum=0.9)
dart_optimizer = optim.SGD(dart_encoder.parameters(), lr=0.01, momentum=0.9)

from torch.nn import CrossEntropyLoss

In [19]:
from collections import Counter

epochs = 3
for e in range( epochs ):
    e_loss = 0
    for batch in tq.tqdm( dataloader ):
        news_data = {
            'input_ids' : batch[0].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[1].to(device).to(torch.int64),
            'attention_mask' : batch[2].to(device).to(torch.int64),
        }
        
        dart_data = {
            'input_ids' : batch[3].to(device).to(torch.int64), # ( B, seq_len )
            'token_type_ids' : batch[4].to(device).to(torch.int64),
            'attention_mask' : batch[5].to(device).to(torch.int64),
        }

        corp_table = batch[6]


        corp_code = corp_table.tolist()
        dic = {}


        batch_size = batch[0].shape[0]
        del batch
        
        news_encoder.train()
        news_encoder.zero_grad()
        news_emb = news_encoder(**news_data) # (B, hidden_dim)
        dart_encoder.train()
        dart_encoder.zero_grad()
        dart_emb = dart_encoder(**dart_data) # (B, hidden_dim)
        sim_score = torch.matmul( news_emb, dart_emb.T ) # (B , B)
        
        target = torch.arange(start=0, end = batch_size).to(device).to(torch.int64)
    
        CE_loss = CrossEntropyLoss()
        loss = CE_loss(sim_score, target)
        e_loss += loss.item()

        loss.backward()
        dart_optimizer.step()
        news_optimizer.step()

        del dart_emb
        del news_emb
        torch.cuda.empty_cache()
    print(e_loss / len(dataloader))


3.8170815729858854



2.764675479714233



2.7640993264642093


6032

# validation

In [33]:
e_loss = 0
news_encoder.eval()
dart_encoder.eval()
num_correct = 0
for batch in tq.tqdm( val_dataloader ):
    news_data = {
        'input_ids' : batch[0].to(device).to(torch.int64), # ( B, seq_len )
        'token_type_ids' : batch[1].to(device).to(torch.int64),
        'attention_mask' : batch[2].to(device).to(torch.int64),
    }
    
    dart_data = {
        'input_ids' : batch[3].to(device).to(torch.int64), # ( B, seq_len )
        'token_type_ids' : batch[4].to(device).to(torch.int64),
        'attention_mask' : batch[5].to(device).to(torch.int64),
    }

    corp_table = batch[6]


    corp_code = corp_table.tolist()
    dic = {} # for pooling for same corp


    batch_size = batch[0].shape[0]
    del batch
    
    with torch.no_grad():
        news_emb = news_encoder(**news_data) # (B, hidden_dim)
        dart_emb = dart_encoder(**dart_data) # (B, hidden_dim)

        sim_score = torch.matmul( news_emb, dart_emb.T ) # (B , B)

        top_idx = torch.argmax(sim_score, dim = 1) # (B)

        
        for p_i, pred in enumerate(top_idx):
            if p_i == pred:
                num_correct += 1
        
        target = torch.arange(start=0, end = batch_size).to(device).to(torch.int64)

        CE_loss = CrossEntropyLoss()
        loss = CE_loss(sim_score, target)
        e_loss += loss.item()
print(e_loss / len(val_dataloader))



2.772235010916105
16.0


In [34]:
print(num_correct / (len(val_dataloader) * BATCH_SIZE))

0.0625
